<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/ml_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Импорт**

In [49]:
from sklearn import tree
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [50]:
from IPython.display import SVG, display, HTML

In [51]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [52]:
from graphviz import Source
from sklearn import tree
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

In [53]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content  coref_df_anns60k.csv  new_anns60k.csv	new_df_anns60k.csv  sample_data
.shortcut-targets-by-id
MyDrive
.file-revisions-by-id
.Trash


In [54]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [55]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv

--2021-05-22 11:05:36--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1162863 (1.1M) [text/plain]
Saving to: ‘coref_df_anns60k.csv.1’

coref_df_anns60k.cs 100%[===================>]   1.11M  --.-KB/s    in 0.07s   

2021-05-22 11:05:36 (16.0 MB/s) - ‘coref_df_anns60k.csv.1’ saved [1162863/1162863]

--2021-05-22 11:05:36--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP reque

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [56]:
# with open('coref_df_anns60k.csv') as csvfile:
with open('new_anns60k.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter='\t')
  with open('new_df_anns60k.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    c = 0
    for row in reader:
      new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
      if len(new_row) > 9:
        new_row = new_row[:9]
      writer.writerow(new_row)


In [57]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('new_df_anns60k.csv', delimiter='\t')

Проверим данные

In [58]:
anns_df

,TOKEN,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,IS_ANSWER,HEAD,NER,COREFERENCE_CLUSTER
0,обсуждаем,1.0,0.0,0.0,0.0,0.0,обсуждаем,0.0,0.0
1,препарат,1.0,0.0,0.0,1.0,0.0,обсуждаем,0.0,0.0
2,рисперидон,1.0,0.0,0.0,2.0,0.0,препарат,1.0,2.0
3,рисполепт,1.0,0.0,0.0,3.0,0.0,рисперидон,1.0,3.0
4,сперидан,1.0,0.0,0.0,4.0,0.0,рисполепт,1.0,4.0
...,...,...,...,...,...,...,...,...,...
15405,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15406,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15407,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15408,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [59]:
import numpy as np

In [60]:
cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k.csv', delimiter=' ')
cor_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
1,препарат,[-0.17725945 0.94834274 0.90867615 0.850609...,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
2,рисперидон,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,[-0.17725945 0.94834274 0.90867615 0.850609...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,0
3,рисполепт,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,[ 0.15259643 1.0673474 0.84192747 -0.077515...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,0
4,сперидан,[ 0.08634768 0.893356 0.55127096 -0.065075...,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.02878256 0.29778534 0.18375699 -0.021691...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26492,и,[ 0.5048941 0.81449467 0.29763442 0.603937...,0,22,68,0,7,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26493,таможенные,[ 0.73242044 -0.24206167 -0.6556247 -0.476650...,0,22,68,0,8,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26494,пошлины,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,0,22,68,0,9,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0
26495,как,[ 1.0349948e+00 7.3793733e-01 -3.8374668e-01 ...,0,22,68,0,10,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0


In [61]:
cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

0.8436395154681279

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

In [62]:
cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]

In [63]:
cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]

In [64]:
cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]

In [65]:
cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]

Добавим размеченные данные, и посмотрим, что получилось.

In [66]:
cor_df['NER'] = anns_df['NER']
cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
# cor_df

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [67]:
df = cor_df.iloc[:8060,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [68]:
len(df.loc[df['NER'] == 1])

358

Из них местоимений (прономинальные анафоры).

In [69]:
len(df.loc[df['NER'] == 3])

70

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [70]:
len(df.loc[df['NER'] == 2])

124

In [71]:
len(df.loc[df['NER'].isin([1,2,3])])

552

Посмотрим на данные.

In [72]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
3,рисполепт,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.012426247, 1.0408522, 0.6913349, -0.1638528...",1.0,3.0
4,сперидан,"[0.08634768, 0.893356, 0.55127096, -0.06507572...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.02878256, 0.29778534, 0.18375699, -0.021691...",1.0,4.0
21,лекарством,"[0.598588, -0.07832722, 0.23088205, -0.3704158...",0,1,1,1,10,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,"[1.3479745, 0.34477097, 0.20343542, 0.10220611...","[0.05498165, -0.00549405, 0.01865345, -0.03142...","[0.42120528, 0.21546456, 0.00441866, -0.240917...",2.0,0.0
142,рисполепта,"[0.20826544, 0.5118653, 0.6703367, -0.22502856...",0,1,1,13,7,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,"[0.5131939, 0.9708054, 0.53676724, 0.09788364,...","[0.11585727, 0.1283627, 0.02299715, -0.0095005...","[0.28126505, 0.3790152, 0.5397224, -0.09368, -...",1.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7998,ибупрофена,"[0.41528076, 0.26439488, 0.2783114, 0.14523816...",1,5,46,0,1796,inan,gent,masc,NaN,NOUN,ROOT,ибупрофена,inan,gent,masc,NOUN,"[0.41528076, 0.26439488, 0.2783114, 0.14523816...","[0.06112305, 0.05345329, 0.06121647, 0.0147250...",[0],1.0,1800.0
8013,ибупрофена,"[0.41528076, 0.26439488, 0.2783114, 0.14523816...",1,5,47,0,1811,inan,gent,masc,NaN,NOUN,nmod,мг,inan,nomn,masc,NOUN,"[0.0851192921, 0.789108872, 0.923557222, -0.23...","[0.0163466763, 0.00905085634, 0.00171076425, 0...","[0.34105581, 0.245229617, 0.321489245, 0.23382...",1.0,1815.0
8014,аскофен,"[0.07804646, 0.12211595, 0.687455, 0.5667267, ...",1,5,47,0,1812,inan,nomn,masc,NaN,NOUN,nsubj,называется,NaN,NaN,NaN,VERB,"[-0.24524373, 0.20882072, 0.11129183, -0.17496...","[0.0163466763, 0.00905085634, 0.00171076425, 0...","[-0.08169067, 0.38310084, 0.4216038, -0.049363...",1.0,1816.0
8057,ибупрофена,"[0.41528076, 0.26439488, 0.2783114, 0.14523816...",1,5,48,0,1855,inan,gent,masc,NaN,NOUN,nmod,мг,inan,nomn,masc,NOUN,"[0.0851192921, 0.789108872, 0.923557222, -0.23...","[0.00764887, -0.00614425, 0.02870062, 0.012436...","[0.34105581, 0.245229617, 0.321489245, 0.23382...",1.0,1859.0


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [73]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
576,зелдокс,"[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",0,2,0,0,2,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.03948287, 0.24082112, 0.22588146, -0.017884...","[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",1.0,2.0
1870,оланзапин,"[-0.082683861, 1.2017089, 0.67815673, -0.22599...",0,3,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01915067, 0.0875298, 0.09080387, 0.00378973...","[-0.082683861, 1.2017089, 0.67815673, -0.22599...",1.0,2.0
5555,галоперидол,"[-0.03019532, 1.2394575, 0.18222564, -0.215572...",0,4,0,0,2,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[-0.03747297, 0.5554195, 0.15133946, -0.011017...","[-0.10372739, 1.0939001, 0.5454509, 0.3175185,...",1.0,2.0
6200,тиаприд,"[0.27581334, 0.80696917, 1.0271364, -0.1037090...",0,5,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.10855111, 0.28903672, 0.2550475, -0.0276597...","[0.27581334, 0.80696917, 1.0271364, -0.1037090...",1.0,3.0


In [74]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [75]:
token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [76]:
df.TOKEN_VECT[710]

array([nan])

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [77]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [78]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

Посмотрим на данные.

In [94]:
df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,0.424228,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0.481209,0.904469,0.564925,0.0,0.0
1,препарат,0.403455,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0.481209,0.904469,0.564925,0.0,0.0
2,рисперидон,0.933164,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,0.987311,0.904469,0.917969,1.0,2.0
3,рисполепт,0.863874,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,0.415625,0.904469,0.856327,1.0,3.0
4,сперидан,0.806514,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,0.438658,0.904469,0.806874,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8055,400,0.410595,1,5,48,0,1853,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0.295454,0.487671,0.687963,0.0,0.0
8056,мг,0.611221,1,5,48,0,1854,inan,nomn,masc,NaN,NOUN,nmod,пп,NaN,NaN,NaN,NaN,0.266755,0.487671,0.609623,0.0,0.0
8057,ибупрофена,0.632453,1,5,48,0,1855,inan,gent,masc,NaN,NOUN,nmod,мг,inan,nomn,masc,NOUN,0.295454,0.487671,0.687963,1.0,1859.0
8058,ибупрофен,0.637485,1,5,48,0,1856,NaN,NaN,masc,NaN,ADJS,obj,\n\n,NaN,NaN,NaN,NaN,0.000000,0.487671,0.725556,0.0,0.0


In [39]:
df.describe()

,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
count,8060.0,8060.000000,8060.000000,8060.000000,8060.000000,8060.000000,8060.0,8060.0,8060.0,8060.000000,8060.000000
mean,0.0,0.942680,3.239082,18.959305,3.981390,1164.631266,0.0,0.0,0.0,0.101241,68.585856
std,0.0,0.232468,1.180049,13.343202,5.343252,1015.556727,0.0,0.0,0.0,0.417002,359.993333
min,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
25%,0.0,1.000000,3.000000,7.000000,1.000000,314.000000,0.0,0.0,0.0,0.000000,0.000000
50%,0.0,1.000000,3.000000,18.000000,2.000000,899.500000,0.0,0.0,0.0,0.000000,0.000000
75%,0.0,1.000000,4.000000,29.000000,5.000000,1755.250000,0.0,0.0,0.0,0.000000,0.000000
max,0.0,1.000000,5.000000,49.000000,31.000000,3668.000000,0.0,0.0,0.0,3.000000,3653.000000


## **Модели. Обучение, оценка.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [81]:
for_factorize = 'ANIMACY	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS']

In [82]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ...,  0, -1,  0])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [95]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'аскофен', 'ибупрофена', 'yurij76', 'называется', 'составе', 'кофеина',
        'ацетилки', 'сказывается', 'кровообращении', 'ибупрофен'],
       dtype='object', length=1576),
 'HEAD_ANIMACY': Index(['inan', 'anim'], dtype='object'),


In [90]:

# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [223]:
train, test = train_test_split(df, test_size=0.2)

In [224]:
train = train.reset_index()

In [225]:
# train = train.iloc[:,:24].drop(['TOKEN', 'level_0', 'index'], axis=1)
# train = train.drop(['TOKEN', 'index'], axis=1)

In [226]:
test = test.reset_index()

In [227]:
test

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,2236,в,0.194644,1,3,5,10,352,-1,-1,-1,-1,7,9,561,0,5,1,1,0.429490,0.468432,0.234387,0.0,0.0
1,1760,паническая,0.314059,1,2,19,1,1095,-1,0,1,-1,10,17,380,0,0,1,1,0.437567,0.526826,0.314198,0.0,0.0
2,2602,все,0.220498,1,3,8,13,718,-1,-1,-1,-1,3,12,655,-1,1,2,8,0.418810,0.565218,0.432229,0.0,0.0
3,5227,v,0.287518,1,3,39,0,3343,-1,-1,-1,-1,-1,28,336,-1,-1,-1,-1,0.000000,0.598076,0.420927,0.0,0.0
4,5256,зипрексы,0.801849,1,3,39,3,3372,1,0,0,-1,1,10,1091,-1,-1,-1,14,0.417195,0.777403,0.804964,1.0,3388.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607,4218,рынок,0.244928,1,3,23,25,2334,0,3,0,-1,1,12,914,-1,-1,-1,0,0.359704,0.375171,0.555419,0.0,0.0
1608,367,p.s,0.256656,1,1,2,6,87,-1,-1,-1,-1,-1,0,155,-1,-1,-1,-1,0.242816,0.425912,0.000000,0.0,0.0
1609,281,подумать,0.419399,1,1,2,0,1,-1,-1,-1,-1,6,8,14,-1,-1,-1,3,0.400718,0.423854,0.466340,0.0,0.0
1610,19,частности,0.343163,0,1,1,1,8,0,4,1,-1,1,15,10,-1,-1,-1,4,0.311518,0.445404,0.356766,0.0,0.0


In [228]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [229]:
test.loc[test['NER'] != 0]

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
4,5256,зипрексы,0.801849,1,3,39,3,3372,1,0,0,-1,1,10,1091,-1,-1,-1,14,0.417195,0.777403,0.804964,1.0,3388.0
6,5923,оно,0.283680,1,4,6,9,299,-1,0,2,1,5,12,1237,-1,-1,-1,0,0.330760,0.495387,0.400351,3.0,367.0
9,5224,нейролептиков,0.714608,1,3,38,5,3340,0,1,0,-1,1,21,1086,-1,0,0,8,0.488264,0.545912,0.722995,2.0,0.0
17,5050,гиперпролактинемия,0.629740,1,3,37,1,3166,0,0,1,-1,1,21,1049,0,1,0,1,0.229309,0.555393,0.695034,1.0,3182.0
47,577,зипрасидон,0.884008,0,2,0,0,3,1,0,0,-1,1,2,233,0,0,0,1,0.398841,0.928780,0.888996,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,5138,рисперидон,0.933164,1,3,37,4,3254,1,0,0,-1,1,10,1077,-1,-1,-1,0,0.406203,0.623602,0.712205,1.0,3270.0
1541,1414,он,0.329561,1,2,14,1,749,-1,0,0,1,5,12,189,-1,-1,-1,0,0.375194,0.643825,0.579305,3.0,829.0
1561,4427,оланзапина,0.798818,1,3,27,0,2543,0,0,1,-1,1,21,965,0,3,0,1,0.398025,0.498245,0.775434,1.0,2559.0
1580,5269,абилифай,0.796579,1,3,41,0,3385,0,0,0,-1,1,0,319,0,0,0,1,0.419986,0.619171,0.535106,1.0,3401.0


Делаем наборы для обучения моделей.

In [252]:
X, y, X_test, y_test = train.drop(['TOKEN', 'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER'], axis=1).iloc[:,:18], \
                      train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

In [253]:
X

,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER
0,0.346189,1,1,6,2,166,-1,2,0,-1,10,7,184,0,0.292213,0.496825,0.612695,2.0
1,0.179801,1,2,3,2,35,-1,-1,-1,-1,7,9,283,1,0.286283,0.347518,0.225133,0.0
2,0.453466,1,3,17,2,1301,-1,-1,-1,-1,6,19,455,0,0.273614,0.551569,0.436864,0.0
3,0.426893,1,3,18,2,1403,-1,2,-1,-1,10,17,621,1,0.414789,0.553312,0.429660,0.0
4,0.179695,1,2,14,5,821,-1,-1,-1,-1,3,5,320,9,0.292515,0.528875,0.383520,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6443,0.214050,1,5,18,1,379,-1,-1,-1,-1,7,9,294,2,0.509207,0.504205,0.379267,0.0
6444,0.211181,1,3,23,17,2185,0,0,1,-1,1,12,848,0,0.324344,0.490031,0.407296,0.0
6445,0.509841,1,3,17,4,1317,-1,2,0,-1,10,7,120,3,0.446725,0.543597,0.538834,0.0
6446,0.480709,1,2,19,1,1136,-1,-1,-1,-1,8,5,393,5,0.329525,0.526826,0.608127,0.0


In [254]:
X_test

,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER
0,0.194644,1,3,5,10,352,-1,-1,-1,-1,7,9,561,1,0.429490,0.468432,0.234387,0.0
1,0.314059,1,2,19,1,1095,-1,0,1,-1,10,17,380,1,0.437567,0.526826,0.314198,0.0
2,0.220498,1,3,8,13,718,-1,-1,-1,-1,3,12,655,8,0.418810,0.565218,0.432229,0.0
3,0.287518,1,3,39,0,3343,-1,-1,-1,-1,-1,28,336,-1,0.000000,0.598076,0.420927,0.0
4,0.801849,1,3,39,3,3372,1,0,0,-1,1,10,1091,14,0.417195,0.777403,0.804964,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607,0.244928,1,3,23,25,2334,0,3,0,-1,1,12,914,0,0.359704,0.375171,0.555419,0.0
1608,0.256656,1,1,2,6,87,-1,-1,-1,-1,-1,0,155,-1,0.242816,0.425912,0.000000,0.0
1609,0.419399,1,1,2,0,1,-1,-1,-1,-1,6,8,14,3,0.400718,0.423854,0.466340,0.0
1610,0.343163,0,1,1,1,8,0,4,1,-1,1,15,10,4,0.311518,0.445404,0.356766,0.0


In [255]:
y

0       442.0
1         0.0
2         0.0
3         0.0
4         0.0
        ...  
6443      0.0
6444      0.0
6445      0.0
6446      0.0
6447      0.0
Name: COREFERENCE_CLUSTER, Length: 6448, dtype: float64

Обучаем модели.

In [256]:
clf = tree.DecisionTreeClassifier(max_depth=200)
clf.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=200, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [257]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=200)
clf_entropy.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=200, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [258]:
clf.score(X_test, y_test), clf_entropy.score(X_test, y_test)

(0.9460297766749379, 0.9466501240694789)

In [259]:
randf_clf = RandomForestClassifier(max_depth=15, n_estimators=600, min_samples_split=4, bootstrap=True, )
randf_clf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=600,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [260]:
regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Функция для оценки моделей

In [261]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [262]:
pred= clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)

In [263]:
my_eval(list(pred), list(y_test))

6  true positive
87  false positive
6  true negative
0  false negative
---------
0.06451612903225806  presicion
1.0  recall
0.12121212121212122  f-score


In [264]:
my_eval(list(pred_randf), list(y_test))

6  true positive
69  false positive
9  true negative
7  false negative
---------
0.08  presicion
0.46153846153846156  recall
0.13636363636363638  f-score


In [265]:
my_eval(list(pred_entropy), list(y_test))

12  true positive
82  false positive
1  true negative
2  false negative
---------
0.1276595744680851  presicion
0.8571428571428571  recall
0.2222222222222222  f-score


In [266]:
my_eval(list(pred_regr), list(y_test))

0  true positive
1612  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score
